# Basic morphological analysis

Alex Sutton and Ted Brookings, Marder lab

This tutorial assumes you have an xml or nml file traced in Knossos (or similar). The file has a list of nodes (x,y,z,radius) that are identified by a node id (int) and a list of edges connecting the nodes. The code can be easily adapted for other generic file types. Knossos is fast, free, lightweight and easy to use. It's available at www.knossostool.org.

Depending on how large your skeleton file is, this process could take a few minutes. The goal is a .hoc file, which consists of nodes that make up segments and a connectivity matrix of those segments. A hoc file is handy because the simulator NEURON (www.neuron.yale.edu) reads them natively and Ted and I developed toolboxes that work easily with hocs. The flow of the pipeline is as follows:

1. Convert xml (nml) file to a hoc
2. Remove accidental loops 
3. Scale the coordinates
4. Load the file as a geometry object
5. Look at example morphological features

For this, you'll need:
* xml (nml) file
* voxel dimensions (of the scan)
* this toolbox



In [3]:
# Import packages
import sys, os # Access to operating system
from XmlToHoc_simple import * # To convert to hoc
from pyramidal_nxRemoveLoops import * # Remove loops
from knossos_scaleCoords import * # Scale coordinates
from pyramidal_readExportedGeometry import * # Load geo object
from pyramidal_getProperties import * # Analyze properties

In [4]:
# 1. Convert to hoc
xmlfile = "878_067_GM_(Adriane).nml"
new_hocfile = "878_067_GM_(Adriane).hoc"
_ = SkelHoc(xmlfile, new_hocfile)

Quantal distance is 64.25730 
Called Refine #3


In [5]:
# 2. Remove accidental loops 
#    This uses NetworkX
hoc_cleaned = "878_067_GM_(Adriane)_noloops.hoc"
_ = rewrite_hoc(new_hocfile, hoc_cleaned)

878_067_GM_(Adriane).hoc has 12888 nodes and 13667 edges
Should have 12887 edges
Predicts ~ 780 loops
Actually found 782 loops
New graph has 0 loops
Num edges: 12885


In [6]:
# 3. Scale the coordinates
voxel = [0.732,0.732,0.488] # (x,y,z)
load_and_fix(hoc_cleaned, voxel)

In [7]:
# 4. Load the geometry object
# I re-named the file to make it a little more compact
scaled = "878_067_GM_(Adriane)_scaled.hoc"
geo = demoReadsilent(scaled)

Number of subgraphs = 3 / size of graphs: [38, 149, 6257]
Number of subgraphs = 3 / size of graphs: [38, 149, 6257]
Removed all but largest subgraphs
self._axonBranches is length: 1
self._axonBranches is length: 1
self._axonBranches is length: 1
self._axonBranches is length: 1
self._axonBranches is length: 1
self._axonBranches is length: 1
self._axonBranches is length: 1
Soma is filament_999[0].
1 axons are:
['filament_999[960]']
From soma to tips, tortuosity is 2.4 +- 1.9


## Geometry object

Here we can see that the final geometry object contains 97.1 % of the nodes included in the original object. This geometry object can be easily parsed to extract meaningful morphological features. Below I show how to do a sanity check on the object, how to extract common features, and then how to work with the object to extract an entirely new feature.

Ted and I developed all of the algorithms here unless noted otherwise in the code (citations are given in the code as well as in the manuscript, Sutton et al., 2016). Some ideas were taken from L-Measure, NeuroMorpho and other morphology sources; these are also cited in the paper. 

In [8]:
# We can examine the file for a little to make sure it looks alright
print('%i nodes, %i segments, %i branches'
      %(len(geo.nodes), len(geo.segments), len(geo.branches)))

6258 nodes, 6257 segments, 1298 branches


In [9]:
# pyramidal_getProperties.py includes functions for many common features
# Many of the features are described at length in Sutton et al. (2016).

# I have a plotting suite that is available for download at
# https://github.com/acsutt0n/PlottingSuite-python-
sys.path.append('/home/alex/PlottingSuite-python-/')
from pretty_plot import *
%matplotlib inline

# Branch angles
bAngles = branch_angles(geo)
circular_hist([bAngles], ['GM'])

ModuleNotFoundError: No module named 'pretty_plot'

In [ ]:
# Tortuosity (computed alongside path length)
paths, torts = path_lengths2(geo)
violin_spline([paths, [0], [0]], ['GM', 'l', 'l'], 
              stepfilled=False, title='Path length')
violin_spline([torts, [0], [0]], ['GM', 'l', 'l'], 
              stepfilled=False, title='Path tortuosity')
# Here I added some empty sets otherwise IPython Notebook
#    fits the width of the page, which looks silly

_Available properties in pyramidal_getProperties.py_

* Branch angle
* Branch point degree (# of daughter branches)
* Path length
* Path tortuosity
* Sholl (use hooser_sholl)
* Partition asymmetry (a/symmetry index)
* Torque (angle between successive bifurcations)
* Neuropil fitting (unbiased space-filling measure)
* Branch length, order, distance to soma and position
* Inter-tip distance (path and Euclidean)
* Fractal dimension (after Caserta et al., 1995)
* Soma position
* Tip rank (path length relative to nodes of similar Euc. distance)
* Rall radius - fitting exponentials

_pyramidal_Subtrees.py_

I have also developed a subtrees module. Subtree analysis involves all branches that are not included in the paths between the soma and the axons (which I call "main paths"). The above analyses and subtree analyses all require geometry objects as their first input. The subtree module also requires Matlab for some of the functions. These are also described in Sutton et al. (2016).

## Other analyses

If you want to examine a novel morphological feature, doing so is easy with geometry objects. Let's say I want to manually identify the axons and get the path length to each of them; this might be something worth doing. 

I'll following this process:
1. Write a function to manually identify axons
2. Find the segment object for each axon
3. Write a function that returns the axon lengths for each supplied axon

In [ ]:
def idAxons(geo, retax=False, show=True):
  """
  Identify axons manually
  """
  btags = []
  for b in geo.branches:
    for k in b.tags:
      if k.split('_')[0] != 'filament':
        btags.append([geo.branches.index(b),k])
  if show:
    for b in btags:
      if b[1] == 'Axon':
        c0, c1 = geo.branches[b[0]].coordAt(0), geo.branches[b[0]].coordAt(1)
        thiscol = color=np.random.random(3)
        plt.plot([c0[0], c1[0]], [c0[1], c1[1]], color=thiscol, lw=3, alpha=0.6)
        plt.text(c1[0], c1[1], b[0], fontsize=10, color=thiscol)
    for s in geo.segments:
      c_ = s.coordAt(0)
      plt.plot(c_[0], c_[1], '.', color='gray', alpha=0.4)
    plt.show()
  if retax:
    return [j[0] for j in btags]
  return

idAxons(geo)

If I wanted to, I could develop this further so a larger variety of potential axons would be identified. I would also use a snapshot of the original scan to make sure what I called an axon was correct. In fact, I do all of this in the module _adriane2.py_, but here we'll assume these axons are good enough. 

I want 375 and 475, which are indices of geo.branches. So simply calling geo.branches[375] gives me the branch I think is an axon. I want to find the farthest segment in that branch, and then find all the segments in the path to that axon.


In [ ]:
def lastSegment(geo, ax_ind):
    # Find the last segment of that axon
    pDF = PathDistanceFinder(geo, geo.soma)
    seg_cands = [s for s in geo.segments if s.name in
                      geo.branches[ax_ind].tags]
    dist0 = pDF.distanceTo(seg_cands[0])
    dist1 = pDF.distanceTo(seg_cands[-1])
    if dist0 > dist1:
        print('Axon %i path: %.2f um' %(ax_ind, dist0))
        return seg_cands[0]
    else:
        print('Axon %i path: %.2f um' %(ax_ind, dist1))
        return seg_cands[-1]

# Now, get the farthest segments and their distances for the axons
ax1 = lastSegment(geo, 375)
ax2 = lastSegment(geo, 475)

In [ ]:
# As measured from the soma.